# Train and Apply Models

In [2]:
from ML.model_training import (
    omit_patient_video,
    single_user_split,
    train_lstm,
    train_random_forest
)
from sklearn.metrics import (
    accuracy_score,
    f1_score,
    precision_score,
    recall_score,
    confusion_matrix,
    classification_report,
)
import numpy as np
import pandas as pd
from ML import utils
import sys
import random
from itertools import product

In [3]:
# remove_list = [0, 1, 2, 4, 5, 6, 8, 9, 10, 11, 14, 16, 17, 18, 19, 22]
remove_list=[]
X_train_vals, X_test_vals, arousal_train_vals, arousal_test_vals = omit_patient_video(
    target="arousal",
    trials=5,
    # selected_user=3,
    exclude_users=remove_list,
    filename="datasets/features_table_psd_only.csv"
)
features = utils.filter_features(X_train_vals.columns, remove_bands=["gamma", "delta"])

def balance(X, y, seed=5):
    c = y.value_counts()
    if c.get("high", 0) == c.get("low", 0):
        return X.reset_index(drop=True), y.reset_index(drop=True)
    maj = c.idxmax()
    m = c.min()
    keep = y[y != maj].index.union(y[y == maj].sample(m, random_state=seed).index)
    return X.loc[keep].reset_index(drop=True), y.loc[keep].reset_index(drop=True)

arousal_train = pd.Series(
    np.where(arousal_train_vals > 3.8, "high", "low"),
    index=arousal_train_vals.index,
    dtype="string",
)
arousal_test = pd.Series(
    np.where(arousal_test_vals > 3.8, "high", "low"),
    index=arousal_test_vals.index,
    dtype="string",
)

# X_train, arousal_train = balance(X_train_vals, arousal_train)
# X_test, arousal_test = balance(X_test_vals, arousal_test)

print("arousal_train counts:\n", arousal_train.value_counts(dropna=False))
print("arousal_test counts:\n", arousal_test.value_counts(dropna=False))

subjects = [1, 3, 12, 15]
# subjects = []
# for _ in range(5):
#     subjects.append(random.randint(0, 22))
# print(subjects)
subjects = [12]

Held-out patient: 18 | Held-out (patient, video) trials: [(18, 0), (18, 11), (18, 13), (18, 16), (18, 17)] | Excluded users: []
arousal_train counts:
 low     21947
high    19952
Name: count, dtype: Int64
arousal_test counts:
 high    392
low      98
Name: count, dtype: Int64


## LSTM CV Optimizer

In [4]:


param_grid = {
    "lr": [0.0001],
    "epochs": [25],
    "units": [128],
    "batch_size": [128],
}

best_params = None
best_mean_acc = -float("inf")

print("Starting global hyperparameter search...\n")

for lr, epochs, units, batch_size in product(
    param_grid["lr"],
    param_grid["epochs"],
    param_grid["units"],
    param_grid["batch_size"],
):
    combo_accs = []
    for i in subjects:
        relabel = False
        while True:
            while True:
                X_train, X_test, arousal_train, arousal_test = omit_patient_video(
                    target="arousal",
                    selected_user=i,
                    trials=18,
                    # holdout_videos=[2, 10, 15],
                    exclude_users=remove_list,
                    filename="datasets/features_table.csv",
                    relabel=relabel,
                )
            

                # binarize labels
                arousal_train = pd.Series(
                    np.where(arousal_train > 3.8, "high", "low"),
                    index=arousal_train.index,
                    dtype="string",
                )
                arousal_test = pd.Series(
                    np.where(arousal_test > 3.8, "high", "low"),
                    index=arousal_test.index,
                    dtype="string",
                )

                # optional class sanity check if you want it back:
                # c = arousal_test.value_counts()
                # if c.get("high", 0) == 0 or c.get("low", 0) == 0:
                #     continue

                # X_train, arousal_train = balance(X_train, arousal_train)
                # X_test, arousal_test = balance(X_test, arousal_test)
                print("arousal_train counts:\n", arousal_train.value_counts(dropna=False))
                print("arousal_test counts:\n", arousal_test.value_counts(dropna=False))
                break

            X_train_sub = X_train.loc[:, features]
            X_test_sub = X_test.loc[:, features]

            lstm, X_test_eval, y_test_eval = train_lstm(
                X_train_sub,
                X_test_sub,
                arousal_train,
                arousal_test,
                lr=lr,
                epochs=epochs,
                units=units,
                batch_size=batch_size,
                dropout=0.5,
                recurrent_dropout=0.2,
                bidirectional=True,
            )
            y_prob = lstm.predict(X_test_eval).ravel()
            arousal_pred = (y_prob >= 0.5).astype(int)

            acc = accuracy_score(y_test_eval, arousal_pred)
            print(acc)
            # if acc < 0.55 and not relabel:
            #     print("attempt relabel")
            #     relabel = True
            # else:
            relabel = False
            break

        combo_accs.append(float(acc))

        print("\nConfusion Matrix (subject):")
        print(confusion_matrix(y_test_eval, arousal_pred))
    mean_acc = float(np.mean(combo_accs))
    print(
        f"Params lr={lr}, epochs={epochs}, units={units}, batch_size={batch_size} "
        f"-> mean acc across subjects = {mean_acc:.4f}"
    )

    if mean_acc > best_mean_acc:
        best_lstm = lstm
        best_mean_acc = mean_acc
        best_params = {
            "lr": lr,
            "epochs": epochs,
            "units": units,
            "batch_size": batch_size,
        }

print("\nBest universal hyperparameters:")
print(best_params)
print(f"Mean accuracy across subjects (tuning): {best_mean_acc:.4f}\n")

Starting global hyperparameter search...

Held-out patient: 12 | Held-out (patient, video) trials: [(12, 0), (12, 1), (12, 2), (12, 3), (12, 4), (12, 5), (12, 6), (12, 7), (12, 8), (12, 9), (12, 10), (12, 11), (12, 12), (12, 13), (12, 14), (12, 15), (12, 16), (12, 17)] | Excluded users: []
arousal_train counts:
 low     21121
high    19425
Name: count, dtype: Int64
arousal_test counts:
 low     924
high    919
Name: count, dtype: Int64
58/58 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step 
0.6592512208355942

Confusion Matrix (subject):
[[577 347]
 [281 638]]
Params lr=0.0001, epochs=25, units=128, batch_size=128 -> mean acc across subjects = 0.6593

Best universal hyperparameters:
{'lr': 0.0001, 'epochs': 25, 'units': 128, 'batch_size': 128}
Mean accuracy across subjects (tuning): 0.6593



### Fine-tune for User

In [7]:
from tensorflow.keras.models import clone_model
from tensorflow.keras import optimizers
import tensorflow as tf
import numpy as np
import pandas as pd

# df = utils.read_table("datasets/features_table_psd_only.csv").reset_index(drop=True)

# df = df[~df["patient_index"].isin([12])].reset_index(drop=True)

# X = df.drop(
#     columns=["patient_index", "video_index", "arousal", "valence", "Unnamed: 0"],
#     errors="ignore",
# )
# X = X.sort_index(axis=1)

# arousal_target = df["arousal"].astype(float)

# features = utils.filter_features(X.columns, remove_bands=["gamma", "delta"])
# X = X.loc[:, features]

# print("Global X shape:", X.shape)

# y_global = (arousal_target > 3.8).astype("int32")

# best_lstm, X_test_eval, y_test_eval = train_lstm(
#     X,
#     None,
#     y_global,
#     None,
#     lr=0.0001,
#     epochs=25,
#     units=128,
#     batch_size=128,
#     dropout=0.5,
#     recurrent_dropout=0.5,
#     bidirectional=True,
#     bidirectional=True,
# )

acc_list = []

for i in range(0, 5):
    X_user_train, X_user_test, arousal_user_train, arousal_user_test = (
        single_user_split(
            target="arousal",
            selected_user=12,
            k_holdouts=5,
            # holdout_videos=[2, 7, 10, 15, 17],
            filename="datasets/features_table_psd_only.csv",
        )
    )

    arousal_user_train = pd.Series(arousal_user_train, dtype="float32")
    arousal_user_test = pd.Series(arousal_user_test, dtype="float32")

    y_user_train = (arousal_user_train > 3.8).astype("int32").to_numpy()
    y_user_test = (arousal_user_test > 3.8).astype("int32").to_numpy()

    X_user_train = X_user_train.sort_index(axis=1)
    X_user_test = X_user_test.sort_index(axis=1)

    X_user_train = X_user_train.loc[:, features]
    X_user_test = X_user_test.loc[:, features]

    X_user_train = X_user_train.to_numpy(dtype="float32")
    X_user_test = X_user_test.to_numpy(dtype="float32")

    X_user_train = X_user_train.reshape(-1, 1, X_user_train.shape[1])
    X_user_test = X_user_test.reshape(-1, 1, X_user_test.shape[1])

    print("User train shape:", X_user_train.shape)
    print("User test shape:", X_user_test.shape)

    user_model = clone_model(best_lstm)
    user_model.build(best_lstm.input_shape)
    user_model.set_weights(best_lstm.get_weights())

    for layer in user_model.layers[:-2]:
        layer.trainable = False

    user_model.compile(
        optimizer=optimizers.Adam(learning_rate=1e-4),
        loss="binary_crossentropy",
        metrics=["accuracy"],
    )

    history = user_model.fit(
        X_user_train,
        y_user_train,
        validation_split=0.2,
        verbose=False,
        batch_size=128,
        epochs=25,
        callbacks=[
            tf.keras.callbacks.EarlyStopping(
                patience=3, restore_best_weights=True, monitor="val_loss"
            )
        ],
    )

    test_loss, test_acc = user_model.evaluate(X_user_test, y_user_test, verbose=0)
    print(f"\nRun {i} - User Test accuracy: {test_acc:.4f}")

    acc_list.append(test_acc)

    y_prob = user_model.predict(X_user_test).ravel()
    y_pred = (y_prob >= 0.5).astype("int32")

    cm = confusion_matrix(y_user_test, y_pred, labels=[0, 1])
    print("\nConfusion Matrix (rows=true, cols=pred):")
    print(cm)

    print("\nOverall accuracy:", accuracy_score(y_user_test, y_pred))

acc_array = np.array(acc_list)
print(f"\nMean user test accuracy over {len(acc_array)} runs: {acc_array.mean():.4f}")
print(f"Std of user test accuracy over {len(acc_array)} runs: {acc_array.std():.4f}")

User train shape: (1411, 1, 98)
User test shape: (432, 1, 98)

Run 0 - User Test accuracy: 0.3843
14/14 ━━━━━━━━━━━━━━━━━━━━ 1s 27ms/step

Confusion Matrix (rows=true, cols=pred):
[[162 178]
 [ 88   4]]

Overall accuracy: 0.38425925925925924
User train shape: (1284, 1, 98)
User test shape: (559, 1, 98)

Run 1 - User Test accuracy: 0.3614
18/18 ━━━━━━━━━━━━━━━━━━━━ 1s 21ms/step

Confusion Matrix (rows=true, cols=pred):
[[  4  90]
 [267 198]]

Overall accuracy: 0.3613595706618962
User train shape: (1365, 1, 98)
User test shape: (478, 1, 98)

Run 2 - User Test accuracy: 0.0209
15/15 ━━━━━━━━━━━━━━━━━━━━ 1s 25ms/step

Confusion Matrix (rows=true, cols=pred):
[[ 10 468]
 [  0   0]]

Overall accuracy: 0.02092050209205021
User train shape: (1439, 1, 98)
User test shape: (404, 1, 98)

Run 3 - User Test accuracy: 0.0891
13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 30ms/step

Confusion Matrix (rows=true, cols=pred):
[[ 29 187]
 [181   7]]

Overall accuracy: 0.0891089108910891
User train shape: (1320, 1, 98)
Us